In [1]:
import torch
import numpy as np
import skimage

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data = np.loadtxt("data/rho20x40_SIMP_Edge.txt", dtype=np.float32)
# [bs, 1, 20, 40]
data = data.reshape(-1,1,40,20).transpose([0,1,3,2])

data_u = np.loadtxt("data/dis20x40_SIMP_Edge.txt", dtype=np.float32)
data_s = np.loadtxt("data/stress20x40_SIMP_Edge.txt", dtype=np.float32)

# filter data
result_label = []
for ind,i in enumerate(data_u):
    if i.max()>5000 or i.min()<-1000:
        pass
    else:
        result_label.append(ind)

data = data[result_label]
data_u = data_u[result_label]
data_s = data_s[result_label]

ref_u0 = torch.from_numpy(data_u).unsqueeze(1).to(device)
ref_uy = ref_u0[:,:,range(1,1722,2)]
ref_ux = ref_u0[:,:,range(0,1722,2)]
ref_s0 = torch.from_numpy(data_s).unsqueeze(1).to(device)
ref_sx = ref_s0[:,:,range(0,2583,3)]
ref_sy = ref_s0[:,:,range(1,2583,3)]
ref_sxy = ref_s0[:,:,range(2,2583,3)]
# [bs, 5, 21, 41]
ref = torch.cat([ref_ux, ref_uy, ref_sx, ref_sy, ref_sxy],1).view(-1,5,41,21).permute(0,1,3,2)

In [3]:
modelname = "experiments/codec/mixed_residual/debug/grf_kle512_ntrain7001_run1_bs64_lr0.001_epochs30000/checkpoints/model_epoch28000.pth"
# modelname = 'experiments/codec/mixed_residual/debug/grf_kle512_ntrain6007_run1_bs64_lr0.001_epochs30000/checkpoints/model_epoch20000.pth'
# modelname = 'experiments/codec/mixed_residual/debug/grf_kle512_ntrain5005_run1_bs256_lr0.001_epochs10000/checkpoints/model_epoch10000.pth'
model = torch.load(modelname)

In [4]:
# sample_num = 15
# num_label = np.arange(0, data.shape[0])
# np.random.shuffle(num_label)
# sample = num_label[:sample_num]

# input = torch.from_numpy(data[sample]).to(device)
input = torch.from_numpy(data).to(device)

In [5]:
output = model(input)

RuntimeError: CUDA out of memory. Tried to allocate 150.00 MiB (GPU 0; 11.91 GiB total capacity; 10.74 GiB already allocated; 54.88 MiB free; 301.18 MiB cached)

In [ ]:
input.shape

In [ ]:
refn0 = ref!=0
print(refn0[0,0].shape)

In [ ]:
ux_max = []
uy_max = []
sx_max = []
sy_max = []
sxy_max = []
for i in range(output.shape[0]):
    ux_max.append(torch.max(torch.abs(((output[i,0,refn0[i,0]] - ref[i,0,refn0[i,0]]) / ref[i,0,refn0[i,0]]))))
    uy_max.append(torch.max(torch.abs((output[i,1,refn0[i,1]] - ref[i,1,refn0[i,1]]) / ref[i,1,refn0[i,1]])))
#     sx_max.append(torch.max(torch.abs(((output[i,2,refn0[i,2]] - ref[i,2,refn0[i,2]]) / ref[i,2,refn0[i,2]]))))
#     sy_max.append(torch.max(torch.abs((output[i,3,refn0[i,3]] - ref[i,3,refn0[i,3]]) / ref[i,3,refn0[i,3]])))
#     sxy_max.append(torch.max(torch.abs(((output[i,4,refn0[i,4]] - ref[i,4,refn0[i,4]]) / ref[i,4,refn0[i,4]]))))

In [ ]:
print(min(ux_max).item())
print(min(uy_max).item())
# print(min(sx_max).item())
# print(min(sy_max).item())
# print(min(sxy_max).item())